# Conversion class

In [3]:
import pandas as pd
import numpy as np
import sys

class eco_2_5_convert:
    def __init__(self, model_file:str, model_name:str,
                EI_3_8 = False):
        """ALLOWED_UINTS: ['cubic meter', 'cubic meter-year', 'EUR', 'EUR2005', 'guest night',
       'hectare', 'hour', 'kg*day', 'kilo Becquerel', 'kilogram',
       'kilometer', 'kilometer-year', 'kilowatt hour', 'litre',
       'megajoule', 'meter', 'meter-year', 'person kilometer', 'Sm3',
       'square meter', 'square meter-year', 'ton kilometer', 'unit']"""
        
        """model_file: the excel model file path
        model_name: a name provided by the user to name the output file and name the model activity
        info_tab: the "info" tab in the eco 2.5 template is a tab where you can add whatever you want
        so it can be used to add calculations and assumptions"""
        
#         
        #Read database ids files
        #ps: make sure all the files are in the same directory you're running jupyter.
        self.activities_raw_ids = pd.read_csv('Activities_id,_Activ_1686047418567_3_9.csv')
        self.categories_default = pd.read_csv('Categories_Latest.csv')
        self.units =  pd.read_csv('Units_updated.csv')
        #Read our LCA custom model
        self.model_activity = pd.read_excel(model_file, sheet_name = 'activity')
        self.model_exchanges = pd.read_excel(model_file, sheet_name = 'exchange')
        self.model_name = model_name
        self.EI_3_8 = EI_3_8
        self.activity_uom = self.model_activity['uom'].iloc[0]

    def convert(self):
        #merge ids with our model to extract all needed ids for the new template
        
        bio_exchange = self.model_exchanges[self.model_exchanges['database'] == 'biosphere3']
        ei_exchange = self.model_exchanges[self.model_exchanges['database'] == 'ecoinvent 3.9']


        bio_exchange_merge = bio_exchange.merge(self.activities_raw_ids, left_on = 'product name',
                                                          right_on = 'products_raw_name')
        ei_exchange_merge = ei_exchange.merge(self.activities_raw_ids,
                                             left_on = ['product name', 'process name', 'location name'],
                 right_on = ['products_raw_name', 'activities_name', 'locations_raw_name'])
        exchanges_merge = pd.concat([bio_exchange_merge, ei_exchange_merge])

        
#         exchanges_merge = self.model_exchanges.merge(self.activities_raw_ids, left_on = ['product name', 'process name', 'location name'],
#                  right_on = ['products_raw_name', 'activities_name', 'locations_raw_name'])
        self.model_activity['raw_uom'] = self.model_activity['uom'].str.lower().str.strip()
        unit_merge = self.model_activity.merge(self.units, left_on = 'raw_uom',
                                              right_on = 'units_raw_name')
        if self.activity_uom not in self.units['units_raw_name'].unique():
            print(f"Activity uom is not of the allowed units please change activity uom to: {units['units_raw_name'].unique()}")
            print("Conversion Aborted")
            sys.exit()
        
        location_merge = self.activities_raw_ids.merge(self.model_activity, left_on = 'locations_raw_name',
                                                       right_on = 'location name')
        new_activity  = pd.read_excel('eco 2.5 model upload template.xlsx',
                                      sheet_name = 'activity')
        category_name = self.model_activity['category name'].iloc[0]
        
        category_id = self.categories_default[self.categories_default['categories_raw_name']==category_name]['categories_id'].iloc[0]
        
        new_activity['Value'] = np.nan
        
        new_activity = new_activity.set_index(['Column','Table'])
        
        new_activity.loc['id', 'source'] = '8b0909fa-a550-11ed-a811-65431102791f'
        
        new_activity.loc['name', 'source'] = 'carbmee custom'
        
        new_activity.loc['name', 'activity'] = self.model_name
        
        new_activity.loc['unit id', 'activity'] = unit_merge['units_id'].iloc[0]
        
        new_activity.loc['unit name', 'activity'] = unit_merge['units_raw_name'].iloc[0]
        
        new_activity.loc['amount', 'activity'] = unit_merge['amount'].iloc[0]
        
        new_activity.loc['location id', 'location'] = location_merge['activities_location_id'].iloc[0]
        
        new_activity.loc['location name', 'location'] = location_merge['locations_raw_name'].iloc[0]
        
        new_activity.loc['name', 'product'] = self.model_activity['product name'].iloc[0]
        
        new_activity.loc['type', 'product'] = self.model_activity['product_type'].iloc[0]
        
        new_activity.loc['name', 'process'] = self.model_activity['process name'].iloc[0]
        
        new_activity.loc['category id', 'product'] = category_id
        
        new_activity.loc['category name', 'product'] = self.model_activity['category name'].iloc[0]
        
        new_activity.loc['description', 'activity'] = self.model_activity['description'].iloc[0]
        
        new_activity.loc['description', 'product'] = self.model_activity['description'].iloc[0]
        
        new_activity.loc['display name en', 'product'] = self.model_activity['product name'].iloc[0]
                
        new_activity.loc['display name en', 'process'] = self.model_activity['process name'].iloc[0]


        new_activity = new_activity.reset_index()
        
            
        type_map = {'biosphere3':'biosphere',
                   'ecoinvent 3.9':'technosphere',
                   'ecoinvent 3.8':'technosphere'}
        exchanges_merge['type'] = exchanges_merge['database'].str.strip().map(type_map) 

        exchanges = exchanges_merge[['activities_id', 'amount', 'products_raw_name', 'processes_raw_name', 'type']].drop_duplicates()
        exchanges = exchanges.rename(columns = {'products_raw_name':'product name',
                                           'processes_raw_name':'process name',
                                               'activities_id':'activity id'})
        
        
        
        writer = pd.ExcelWriter('ECO_2_5_{}_model.xlsx'.format(self.model_name), engine='xlsxwriter')
        
        new_activity.to_excel(writer, sheet_name='activity', index = False)
        
        exchanges.to_excel(writer, sheet_name='exchanges', index = False)
        
        writer.close()
        
        return new_activity, exchanges

In [4]:
# %%time
#call class with inputs
eco_25 = eco_2_5_convert('Models_beverage carton.xlsx', 'beverage carton')

#get outputs from conversion functions
new_activity, exchanges = eco_25.convert()

/Users/burcugulsen/anaconda3/lib/python3.9/site-packages/openpyxl/worksheet/_read_only.py:79: UserWarning: Data Validation extension is not supported and will be removed
  for idx, row in parser.parse():


In [3]:
new_activity

,Column,Table,Value
0,id,source,5b1acea4-a550-11ed-a811-65431102791f
1,name,source,NaN
2,name,activity,"transport, passenger car, medium size, diesel,..."
3,unit id,activity,893323a5-073c-4acb-8453-9a88473d3387
4,unit name,activity,kilometer
5,amount,activity,1
6,description,activity,NaN
7,default waste activity id,activity,NaN
8,default waste product name,activity,NaN
9,default waste process name,activity,NaN


In [4]:
exchanges

,activity id,amount,product name,process name,type
0,1198e8b5-7d0d-44d5-8825-218f80d07e8c,0.174735,"Carbon dioxide, fossil (air)",NaN,biosphere
1,a49d4996-34b4-4693-9bf1-a974f83d44a4,0.000061,"Carbon monoxide, fossil (air, urban air close ...",NaN,biosphere
2,8e71e916-aaa1-45ce-aa58-b1e034b547d5,0.000003,"Dinitrogen monoxide (air, urban air close to g...",NaN,biosphere
